In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
use_wandb = False

In [2]:
if use_wandb:
    import wandb
    wandb.login()

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary

In [4]:
from datasets.cifar10 import trainloader, testloader
from utils.general_utils import setup_env
from utils.training import train_model
from utils.testing import test_model
from models.model_v3 import Net

Files already downloaded and verified
Total number of images:  50000
Mean =  (0.4890062, 0.47970363, 0.47680542) 
 STD =  (0.264582, 0.258996, 0.25643882)
Files already downloaded and verified
Files already downloaded and verified
Batch Size -- 512


In [5]:
cuda, device = setup_env()

train_dataloader, test_dataloader = trainloader, testloader

model = Net(
    batch_norm=True,
    dropout_value=0.0,
).to(device)
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
              ReLU-2           [-1, 32, 32, 32]               0
       BatchNorm2d-3           [-1, 32, 32, 32]              64
           Dropout-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          18,432
              ReLU-6           [-1, 64, 32, 32]               0
       BatchNorm2d-7           [-1, 64, 32, 32]             128
           Dropout-8           [-1, 64, 32, 32]               0
        conv_block-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 32, 16, 16]          18,464
           Conv2d-11           [-1, 32, 16, 16]           1,056
             ReLU-12           [-1, 32, 16, 16]               0
      BatchNorm2d-13           [-1, 32, 16, 16]              64
 transition_block-14           [-1, 32,

In [6]:
if use_wandb:

    wandb.init(
        project='s7_cifar10',              ## unique to each project / assignment
        entity='weights_heist_eva7',       ## this will not change
        ## below are optional but recommended 
        tags=['muhsin'],                   ## can help later to filter runs
        name='version 5 C3 increase',    ## a name for the run. 
        notes="C3 conv >> dilated conv"
    )

In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = StepLR(optimizer, step_size=12, gamma=0.6)

logs = []
EPOCHS = 25
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train_batch_loss, train_batch_acc = train_model(
        model, device, train_dataloader, optimizer, epoch
    )
    test_loss, test_acc = test_model(model, device, test_dataloader)
    scheduler.step()
    
    temp_log = {
        'train_losses': np.mean(train_batch_loss),
        'test_losses': test_loss,
        'train_acc': np.mean(train_batch_acc),
        'test_acc': test_acc,
        "lr": optimizer.param_groups[0]['lr'],
    }
    logs.append(temp_log)
    
    if use_wandb:
        wandb.log(temp_log)

In [8]:
if use_wandb:
    wandb.finish()